In [1]:
import io
import re
import json
import concurrent
import pandas as pd
from datetime import datetime
from google.cloud import storage
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

In [2]:
pd.set_option('max_colwidth', 999999)
pd.options.mode.copy_on_write = True
iopub_data_rate_limit = 1.0e10

In [3]:
# define your regex pattern here to get specific data
def extract_ip_from_log(log_line):
  match = re.search(r'production\.INFO: (.+)$', log_line)
  if match:
      json_part = match.group(1)
      try:
          json_data = json.loads(json_part)
          if 'ip' in json_data:
              return json_data['ip']
      except json.JSONDecodeError:
          pass
  return None

In [4]:
def process_file_chunk(chunk):
    json_list = []
    for line in chunk:
        json_part = extract_ip_from_log(line)
        if json_part:
            json_list.append(json_part)
    return json_list

In [ ]:
bucket_name = "laravel-cron-prod"
source = "laravel-2024-02-25.log" # gs uri

# Explicitly use service account credentials by specifying the private key file.
storage_client = storage.Client.from_service_account_json('/content/perqara-data-532572ce4996.json')

# Get the bucket and blob objects
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(source)

# Download the contents of the blob as a string
txt_data = blob.download_as_text()

In [ ]:
# Create an in-memory file-like object
file_like_object = io.StringIO(txt_data)

with file_like_object as file:
  lines = file.readlines()

In [ ]:
# def search_for_urls_in_lines(lines):
#   urls = set(["https://hera.perqara.com/api/auth/login", "https://hera.perqara.com/api/auth/logout"])
#   result = []

#   for line in lines:
#     # Remove timestamp and INFO part using regex
#     clean_line = re.sub(r'^\[\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\] production\.INFO: ', '', line)

#     if any(url in clean_line for url in urls):
#       result.append(clean_line.strip())

#   return result

In [ ]:
# def search_for_urls_in_lines(lines):
#   urls = set(["https://hera.perqara.com/api/auth/login", "https://hera.perqara.com/api/auth/logout"])
#   results = []
#   for line in lines:
#     if any(url in line for url in urls):
#       match = re.search(r"\[(.*?)\] (.*?): (.*)", line)
#       if match:
#         timestamp, level, message = match.groups()
#         timestamp = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
#         results.append({"timestamp": timestamp.isoformat(), "level": level, "message": message})
#   return results

In [ ]:
def search_for_urls_in_lines(lines):
  urls = set(["https://hera.perqara.com/api/auth/login", "https://hera.perqara.com/api/auth/logout"])
  results = []
  for line in lines:
    line = re.sub(r'^".*"$', '', line)
    if any(url in line for url in urls):
      match = re.search(r"\[(.*?)\] (.*?): (.*)", line)
      if match:
        timestamp, level, message = match.groups()
        timestamp = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
        results.append({"timestamp": timestamp.isoformat(), "level": level, "message": message})
  return results

In [ ]:
max_threads = 2

# Use ThreadPoolExecutor to process lines concurrently
with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
  results = list(executor.map(search_for_urls_in_lines, [lines]))

# Flatten and write the results
flattened_results = [item for sublist in results for item in sublist]

In [ ]:
# OPTIONAL: USED IF WE WANNA INGEST DIRECTLY TO BIGQUERY
# Write the results to a newline-delimited JSON file
with open("search_results.json", "w", encoding="utf-8") as outfile:
  for line in flattened_results:
    # Convert the line to a Python object (if necessary)
    if isinstance(line, str):
      # Skip strings as they're already JSON-formatted
      outfile.write(line + '\n')
    else:
      # Use json.dumps to explicitly avoid backslashes
      json_line = json.dumps(line, ensure_ascii=False, separators=(',', ':'))
      outfile.write(json_line + '\n')

In [ ]:
df = pd.DataFrame(flattened_results)

In [ ]:
df

,timestamp,level,message
0,2024-02-18T00:00:09,production.INFO,"{""type"":""request"",""ip"":""125.160.140.26"",""method"":""POST"",""url"":""https://hera.perqara.com/api/auth/login"",""headers"":{""User-Agent"":""okhttp/4.9.0"",""Accept-Encoding"":""gzip"",""Connection"":""Keep-Alive"",""Host"":""hera.perqara.com"",""Content-Length"":""96"",""Content-Type"":""application/json; charset=UTF-8"",""X-Platform"":""WEB"",""Accept"":""application/json"",""Authorization"":""xxxxxx""},""body"":""{\""password\"":\""@1Mei2002?\"",\""password_confirmation\"":\""@1Mei2002?\"",\""username\"":\""ajahsanam8@gmail.com\""}""}"
1,2024-02-18T00:00:35,production.INFO,"{""type"":""request"",""ip"":""114.125.100.65"",""method"":""DELETE"",""url"":""https://hera.perqara.com/api/auth/logout"",""headers"":{""Accept-Language"":""en-US,en;q=0.9,id-ID;q=0.8,id;q=0.7"",""Accept-Encoding"":""gzip, deflate, br"",""Referer"":""https://perqara.com/"",""Sec-Fetch-Dest"":""empty"",""Sec-Fetch-Mode"":""cors"",""Sec-Fetch-Site"":""same-site"",""Origin"":""https://perqara.com"",""Sec-Ch-Ua-Platform"":""\""Android\"""",""User-Agent"":""Mozilla/5.0 (Linux; Android 6.0.1; vivo 1606) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Mobile Safari/537.36"",""Accept"":""application/json, text/plain, */*"",""Content-Type"":""application/json"",""X-Platform"":""WEB"",""Authorization"":""xxxxxx"",""Sec-Ch-Ua-Mobile"":""?1"",""X-Language"":""id"",""Sec-Ch-Ua"":""\""Chromium\"";v=\""106\"", \""Google Chrome\"";v=\""106\"", \""Not;A=Brand\"";v=\""99\"""",""Content-Length"":""36"",""Connection"":""keep-alive"",""Host"":""hera.perqara.com""},""body"":""{\""fcm_token\"":\""\"",\""device_type\"":\""web\""}""}"
2,2024-02-18T00:05:08,production.INFO,"{""type"":""request"",""ip"":""182.253.54.204"",""method"":""POST"",""url"":""https://hera.perqara.com/api/auth/login"",""headers"":{""X-Platform"":""WEB"",""Accept-Encoding"":""br;q=1.0, gzip;q=0.9, deflate;q=0.8"",""Accept-Language"":""en-ID;q=1.0, id-ID;q=0.9"",""User-Agent"":""Perqara - Clients/1.2.0 (com.limaventura.perqara; build:22; iOS 17.2.1) Alamofire/5.8.0"",""Accept"":""application/json"",""Connection"":""keep-alive"",""Cookie"":""laravel_session=hQlaekSfY84MLdsL7f72G3agZKO4UOJ3ZKs7pOAI"",""Content-Length"":""93"",""Content-Type"":""application/json"",""Host"":""hera.perqara.com""},""body"":""{\""password\"":\""Kelapagad1ng\"",\""username\"":\""6281282861428\"",\""password_confirmation\"":\""Kelapagad1ng\""}""}"
3,2024-02-18T00:56:08,production.INFO,"{""type"":""request"",""ip"":""182.3.53.185"",""method"":""DELETE"",""url"":""https://hera.perqara.com/api/auth/logout"",""headers"":{""User-Agent"":""okhttp/4.9.0"",""Accept-Encoding"":""gzip"",""Connection"":""Keep-Alive"",""Host"":""hera.perqara.com"",""Content-Length"":""389"",""Content-Type"":""application/json; charset=UTF-8"",""X-Platform"":"": ANDROID-LAWYER"",""Accept"":"": application/json"",""Authorization"":""xxxxxx""},""body"":""{\""device_id\"":\""android-e8ALrsavQ6apfjSmJfHcI9:APA91bHAap7ckWmYkuPvz0iAN7H8f9RKz58fk1pre3hBPrfm2vyrzr5DvOKtrQm5rm9xvIpwHo26zv1wsYxxicyNcvWYi_4TkCewFFpkUgbeSDURw1TEMURNro-OD5ibNDJBL0X5HEw_\"",\""device_type\"":\""android\"",\""fcm_token\"":\""e8ALrsavQ6apfjSmJfHcI9:APA91bHAap7ckWmYkuPvz0iAN7H8f9RKz58fk1pre3hBPrfm2vyrzr5DvOKtrQm5rm9xvIpwHo26zv1wsYxxicyNcvWYi_4TkCewFFpkUgbeSDURw1TEMURNro-OD5ibNDJBL0X5HEw_\""}""}"
4,2024-02-18T01:05:28,production.INFO,"{""type"":""request"",""ip"":""114.122.20.247"",""method"":""DELETE"",""url"":""https://hera.perqara.com/api/auth/logout"",""headers"":{""User-Agent"":""okhttp/4.9.0"",""Accept-Encoding"":""gzip"",""Connection"":""Keep-Alive"",""Host"":""hera.perqara.com"",""Content-Length"":""389"",""Content-Type"":""application/json; charset=UTF-8"",""X-Platform"":"": ANDROID-LAWYER"",""Accept"":"": application/json"",""Authorization"":""xxxxxx""},""body"":""{\""device_id\"":\""android-eSQs_v9RTWWiyshjAAN2Vj:APA91bE72GBLp2AHCWwW2Nh2Gll1mF5qr95NjG-F9XNU_dbh2A7dZIMdRiQ9zAy9mEzQouh75ux3Yjc9h7LPeLyy7mn5Pd

In [ ]:
def extract_value(message, key_name):
  try:
    parsed_message = json.loads(message)
    return parsed_message.get(key_name, None)
  except:
    return None  # Handle potential exceptions, like parsing errors

In [ ]:
column_list = ['type', 'ip', 'method', 'url', 'headers', 'body']

for column in column_list:
  df[column] = df['message'].apply(lambda x: extract_value(x, column) or None)

In [ ]:
df

,timestamp,level,message,type,ip,method,url,headers,body
0,2024-02-18T00:00:09,production.INFO,"{""type"":""request"",""ip"":""125.160.140.26"",""method"":""POST"",""url"":""https://hera.perqara.com/api/auth/login"",""headers"":{""User-Agent"":""okhttp/4.9.0"",""Accept-Encoding"":""gzip"",""Connection"":""Keep-Alive"",""Host"":""hera.perqara.com"",""Content-Length"":""96"",""Content-Type"":""application/json; charset=UTF-8"",""X-Platform"":""WEB"",""Accept"":""application/json"",""Authorization"":""xxxxxx""},""body"":""{\""password\"":\""@1Mei2002?\"",\""password_confirmation\"":\""@1Mei2002?\"",\""username\"":\""ajahsanam8@gmail.com\""}""}",request,125.160.140.26,POST,https://hera.perqara.com/api/auth/login,"{'User-Agent': 'okhttp/4.9.0', 'Accept-Encoding': 'gzip', 'Connection': 'Keep-Alive', 'Host': 'hera.perqara.com', 'Content-Length': '96', 'Content-Type': 'application/json; charset=UTF-8', 'X-Platform': 'WEB', 'Accept': 'application/json', 'Authorization': 'xxxxxx'}","{""password"":""@1Mei2002?"",""password_confirmation"":""@1Mei2002?"",""username"":""ajahsanam8@gmail.com""}"
1,2024-02-18T00:00:35,production.INFO,"{""type"":""request"",""ip"":""114.125.100.65"",""method"":""DELETE"",""url"":""https://hera.perqara.com/api/auth/logout"",""headers"":{""Accept-Language"":""en-US,en;q=0.9,id-ID;q=0.8,id;q=0.7"",""Accept-Encoding"":""gzip, deflate, br"",""Referer"":""https://perqara.com/"",""Sec-Fetch-Dest"":""empty"",""Sec-Fetch-Mode"":""cors"",""Sec-Fetch-Site"":""same-site"",""Origin"":""https://perqara.com"",""Sec-Ch-Ua-Platform"":""\""Android\"""",""User-Agent"":""Mozilla/5.0 (Linux; Android 6.0.1; vivo 1606) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Mobile Safari/537.36"",""Accept"":""application/json, text/plain, */*"",""Content-Type"":""application/json"",""X-Platform"":""WEB"",""Authorization"":""xxxxxx"",""Sec-Ch-Ua-Mobile"":""?1"",""X-Language"":""id"",""Sec-Ch-Ua"":""\""Chromium\"";v=\""106\"", \""Google Chrome\"";v=\""106\"", \""Not;A=Brand\"";v=\""99\"""",""Content-Length"":""36"",""Connection"":""keep-alive"",""Host"":""hera.perqara.com""},""body"":""{\""fcm_token\"":\""\"",\""device_type\"":\""web\""}""}",request,114.125.100.65,DELETE,https://hera.perqara.com/api/auth/logout,"{'Accept-Language': 'en-US,en;q=0.9,id-ID;q=0.8,id;q=0.7', 'Accept-Encoding': 'gzip, deflate, br', 'Referer': 'https://perqara.com/', 'Sec-Fetch-Dest': 'empty', 'Sec-Fetch-Mode': 'cors', 'Sec-Fetch-Site': 'same-site', 'Origin': 'https://perqara.com', 'Sec-Ch-Ua-Platform': '""Android""', 'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0.1; vivo 1606) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Mobile Safari/537.36', 'Accept': 'application/json, text/plain, */*', 'Content-Type': 'application/json', 'X-Platform': 'WEB', 'Authorization': 'xxxxxx', 'Sec-Ch-Ua-Mobile': '?1', 'X-Language': 'id', 'Sec-Ch-Ua': '""Chromium"";v=""106"", ""Google Chrome"";v=""106"", ""Not;A=Brand"";v=""99""', 'Content-Length': '36', 'Connection': 'keep-alive', 'Host': 'hera.perqara.com'}","{""fcm_token"":"""",""device_type"":""web""}"
2,2024-02-18T00:05:08,production.INFO,"{""type"":""request"",""ip"":""182.253.54.204"",""method"":""POST"",""url"":""https://hera.perqara.com/api/auth/login"",""headers"":{""X-Platform"":""WEB"",""Accept-Encoding"":""br;q=1.0, gzip;q=0.9, deflate;q=0.8"",""Accept-Language"":""en-ID;q=1.0, id-ID;q=0.9"",""User-Agent"":""Perqara - Clients/1.2.0 (com.limaventura.perqara; build:22; iOS 17.2.1) Alamofire/5.8.0"",""Accept"":""application/json"",""Connection"":""keep-alive"",""Cookie"":""laravel_session=hQlaekSfY84MLdsL7f72G3agZKO4UOJ3ZKs7pOAI"",""Content-Length"":""93"",""Content-Type"":""application/json"",""Host"":""hera.perqara.com""},""body"":""{\""password\"":\""Kelapagad1ng\"",\""username\"":\""6281282861428\"",\""password_confirmation\"":\""Kelapagad1ng\""}""}",request,182.253.54.204,POST,https://hera.perqara.com/api/auth/login,"{'X-Platform': 'WEB', 'Accept-Encoding': 'br;q=1.0, gzip;q=0.9, deflate;q=0.8', 'Accept

In [ ]:
# Masking values containing "password" in the column 'column1'
mask = df['body'].str.contains('password')
df.loc[mask, 'body'] = '***masked***'

In [ ]:
df[df['body'].str.contains('masked')]

,timestamp,level,message,type,ip,method,url,headers,body
0,2024-02-18T00:00:09,production.INFO,"{""type"":""request"",""ip"":""125.160.140.26"",""method"":""POST"",""url"":""https://hera.perqara.com/api/auth/login"",""headers"":{""User-Agent"":""okhttp/4.9.0"",""Accept-Encoding"":""gzip"",""Connection"":""Keep-Alive"",""Host"":""hera.perqara.com"",""Content-Length"":""96"",""Content-Type"":""application/json; charset=UTF-8"",""X-Platform"":""WEB"",""Accept"":""application/json"",""Authorization"":""xxxxxx""},""body"":""{\""password\"":\""@1Mei2002?\"",\""password_confirmation\"":\""@1Mei2002?\"",\""username\"":\""ajahsanam8@gmail.com\""}""}",request,125.160.140.26,POST,https://hera.perqara.com/api/auth/login,"{'User-Agent': 'okhttp/4.9.0', 'Accept-Encoding': 'gzip', 'Connection': 'Keep-Alive', 'Host': 'hera.perqara.com', 'Content-Length': '96', 'Content-Type': 'application/json; charset=UTF-8', 'X-Platform': 'WEB', 'Accept': 'application/json', 'Authorization': 'xxxxxx'}",***masked***
2,2024-02-18T00:05:08,production.INFO,"{""type"":""request"",""ip"":""182.253.54.204"",""method"":""POST"",""url"":""https://hera.perqara.com/api/auth/login"",""headers"":{""X-Platform"":""WEB"",""Accept-Encoding"":""br;q=1.0, gzip;q=0.9, deflate;q=0.8"",""Accept-Language"":""en-ID;q=1.0, id-ID;q=0.9"",""User-Agent"":""Perqara - Clients/1.2.0 (com.limaventura.perqara; build:22; iOS 17.2.1) Alamofire/5.8.0"",""Accept"":""application/json"",""Connection"":""keep-alive"",""Cookie"":""laravel_session=hQlaekSfY84MLdsL7f72G3agZKO4UOJ3ZKs7pOAI"",""Content-Length"":""93"",""Content-Type"":""application/json"",""Host"":""hera.perqara.com""},""body"":""{\""password\"":\""Kelapagad1ng\"",\""username\"":\""6281282861428\"",\""password_confirmation\"":\""Kelapagad1ng\""}""}",request,182.253.54.204,POST,https://hera.perqara.com/api/auth/login,"{'X-Platform': 'WEB', 'Accept-Encoding': 'br;q=1.0, gzip;q=0.9, deflate;q=0.8', 'Accept-Language': 'en-ID;q=1.0, id-ID;q=0.9', 'User-Agent': 'Perqara - Clients/1.2.0 (com.limaventura.perqara; build:22; iOS 17.2.1) Alamofire/5.8.0', 'Accept': 'application/json', 'Connection': 'keep-alive', 'Cookie': 'laravel_session=hQlaekSfY84MLdsL7f72G3agZKO4UOJ3ZKs7pOAI', 'Content-Length': '93', 'Content-Type': 'application/json', 'Host': 'hera.perqara.com'}",***masked***
5,2024-02-18T01:11:01,production.INFO,"{""type"":""request"",""ip"":""103.14.111.238"",""method"":""POST"",""url"":""https://hera.perqara.com/api/auth/login"",""headers"":{""User-Agent"":""okhttp/4.9.0"",""Accept-Encoding"":""gzip"",""Connection"":""Keep-Alive"",""Host"":""hera.perqara.com"",""Content-Length"":""91"",""Content-Type"":""application/json; charset=UTF-8"",""X-Platform"":""WEB"",""Accept"":""application/json"",""Authorization"":""xxxxxx""},""body"":""{\""password\"":\""novaogell27\"",\""password_confirmation\"":\""novaogell27\"",\""username\"":\""6285907492434\""}""}",request,103.14.111.238,POST,https://hera.perqara.com/api/auth/login,"{'User-Agent': 'okhttp/4.9.0', 'Accept-Encoding': 'gzip', 'Connection': 'Keep-Alive', 'Host': 'hera.perqara.com', 'Content-Length': '91', 'Content-Type': 'application/json; charset=UTF-8', 'X-Platform': 'WEB', 'Accept': 'application/json', 'Authorization': 'xxxxxx'}",***masked***
6,2024-02-18T01:11:11,production.INFO,"{""type"":""request"",""ip"":""103.14.111.238"",""method"":""POST"",""url"":""https://hera.perqara.com/api/auth/login"",""headers"":{""User-Agent"":""okhttp/4.9.0"",""Accept-Encoding"":""gzip"",""Connection"":""Keep-Alive"",""Host"":""hera.perqara.com"",""Content-Length"":""103"",""Content-Type"":""application/json; charset=UTF-8"",""X-Platform"":""WEB"",""Accept"":""application/json"",""Authorization"":""xxxxxx""},""body"":""{\""password\"":\""novaogell27\"",\""password_confirmation\"":\""novaogell27\"",\""username\"":\""novamurdiyana27@gmail.com\""}""}",request,103.14.111.238,POST,https://hera.perqara.com/api/auth/login,"{'User-Agent': 'okhttp/4.9.0', 'Accept-Encoding': 'gzip', 'Connection': 'Keep-Alive', 'H